In [1]:
from nn.blocks.residuals import ResidualAdd
import torch.nn as nn
from collections import OrderedDict
import torch
from torch import Tensor
from torchsummary import summary

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
x = torch.rand(1, 3,224,224)

In [18]:
class PreActBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, conv:nn.Module = nn.Conv2d, activation: nn.Module = nn.ReLU, *args, **kwargs):
        super().__init__()
        self.block = nn.Sequential(OrderedDict({
            'bn': nn.BatchNorm2d(in_features),
            'act': activation(),
            'conv': conv(in_features, out_features, *args, **kwargs)
        }))
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.block(x)
        return x
    
class ConvBnBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, conv:nn.Module = nn.Conv2d, *args, **kwargs):
        super().__init__()
        self.block = nn.Sequential(OrderedDict({
            'conv': conv(in_features, out_features, *args, **kwargs),
            'bn': nn.BatchNorm2d(out_features)
        }))
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.block(x)
        return x
   
class ConvBnActBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, conv:nn.Module = nn.Conv2d, activation: nn.Module = nn.ReLU, *args, **kwargs):
        super().__init__()
        self.block = nn.Sequential(OrderedDict({
            'conv': conv(in_features, out_features, *args, **kwargs),
            'bn': nn.BatchNorm2d(out_features),
            'act': activation(),
        }))
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.block(x)
        return x
    


In [19]:
PreActBlock(32, 64, kernel_size=3)

ConvBnBlock(32, 64, kernel_size=3)

ConvBnActBlock(32, 64, kernel_size=3)

ConvBnActBlock(
  (block): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
  )
)

In [4]:
from torchvision.models import densenet121


densenet121(False)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [9]:
%%timeit
from torchvision.models import resnet101

model = resnet101(False)
model.eval()


for i in range(10):
    _ = model(x)

1.61 s ± 35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
from nn.resnet import ResNet
model = ResNet.resnet101()
model.eval()

for i in range(10):
    _ = model(x)

1.97 s ± 50 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
from nn.resnet import ResNet
model = ResNet.resnet101()
model.eval()
summary(model.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
            Conv2d-7           [-1, 64, 56, 56]          36,864
       BatchNorm2d-8           [-1, 64, 56, 56]             128
            Conv2d-9          [-1, 256, 56, 56]          16,384
      BatchNorm2d-10          [-1, 256, 56, 56]             512
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
    ResNetShorcut-13          [-1, 256, 56, 56]               0
             ReLU-14          [-1, 256,

In [73]:
from torchvision.models import resnet101
model = resnet101(False)
model.eval()
summary(model.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [82]:
from nn.senet import SENetBasicBlock

In [88]:
from nn.resnet import ResNet


senet18 = ResNet(block=SENetBasicBlock, deepths=[2, 2, 2, 2])
senet18(x).shape

torch.Size([1, 1000])

In [89]:
from nn.senet import se_resnet18

In [92]:
se_resnet18()

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): SENetBasicBlock(
            (block): Sequential(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (se): SEModule(
          